In [2]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from typing import List, Callable, Dict
from realgam.quantlib import general_utils as gu
from realgam.quantlib.factor_lib.technicals import indicators_cal as ic
from realgam.quantlib.factor_lib.technicals import alpha_101 as alpha101
import datetime
import os
import time
PROJECT_PATH = os.getenv('QuantSystemMVP')
DATA_PATH = f'{PROJECT_PATH}/Data/stock_hist.obj'


In [3]:
class CandlesSeries(pd.Series):
    @property
    def _constructor(self):
        return CandlesSeries

    @property
    def _constructor_expanddim(self):
        return CandlesDataFrame

class CandlesDataFrame(DataFrame):

    # def __init__(self, candles_df):
    #     candles_df = candles_df
    @property
    def _constructor(self):
        return CandlesDataFrame

    @property
    def _constructor_sliced(self):
        return CandlesSeries

    def cross_rank(self, column_to_rank: str, **kwargs) -> pd.Series:

        return self.groupby('date')[column_to_rank].rank(**kwargs)

    def cross_pctrank(self, column_to_rank: str, **kwargs) -> pd.Series:

        return self.groupby('date')[column_to_rank].rank(pct=True, **kwargs)

    def engineer(self, func: Callable, engineer_col: str, groupby_cols = 'ticker', **kwargs) -> pd.Series:

        return self.groupby(groupby_cols)[engineer_col].apply(func, **kwargs)


    def s_engineer(self, func: Callable, engineer_col: str, **kwargs) -> pd.Series:

        return self[engineer_col].apply(func, **kwargs)

    def calc_alpha(self, alpha_func: Callable, args=(), **kwargs) -> pd.Series:

        return alpha_func(self, **kwargs)

In [37]:
stocks_df, stocks_extended_df, available_tickers = gu.load_file(DATA_PATH)

In [ ]:
(-1 * correlation(rank(delta(log(volume), 2)), rank(((close - open) / open)), 6))

In [43]:
stacked_hist = stocks_df.copy()

In [44]:
stacked_hist = CandlesDataFrame(stacked_hist)

In [42]:
s_time_chunk = time.time()
stacked_hist['alpha1'] = stacked_hist.calc_alpha(alpha101.alpha_1, n_ts_arg_max=5, n_std=22)
e_time_chunk = time.time()

print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 38.05243492126465 sec


In [ ]:
n_delta = 2

# stacked_hist[f'volume_delta{n_delta}'] = stacked_hist.groupby('ticker').volume.apply(ic.delta_volume, n=n_delta)
# stacked_hist['ret_intraday'] = stacked_hist.closeadj / stacked_hist.openadj - 1

stacked_hist[f'volume_delta{n_delta}'] = stacked_hist.engineer(ic.delta_volume, 'volume', n=n_delta)
stacked_hist['ret_intraday'] = ic.pct_change(stacked_hist.closeadj, stacked_hist.openadj)

stacked_hist['rank_volumedelta'] = stacked_hist.cross_pctrank(f'volume_delta{n_delta}')
stacked_hist['rank_ret_intraday'] = stacked_hist.cross_pctrank('ret_intraday')

n_roll = 6
s_time_chunk = time.time()
stacked_hist['corr_ranks'] = stacked_hist.groupby('ticker')['rank_volumedelta'].rolling(n_roll).corr(stacked_hist['rank_ret_intraday']).values
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")
stacked_hist['alpha'] = -1 * stacked_hist.corr_ranks


In [ ]:
stacked_hist

In [7]:
A = stacked_hist[stacked_hist.ticker == 'A']

In [9]:
A

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,volume_delta2,ret_intraday,rank_volumedelta,rank_ret_intraday
date,,,,,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0,NaN,0.017857,NaN,0.811883
2012-01-04,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0,NaN,0.001107,NaN,0.573025
2012-01-05,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0,0.498527,0.031215,0.822699,0.874418
2012-01-06,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0,0.012721,0.008358,0.488345,0.778275
2012-01-09,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0,-0.434874,0.020202,0.264791,0.863145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-07,A,133.55,133.550,128.430,130.34,133.550000,133.550000,128.430000,130.340,2376454.0,-0.413138,-0.024036,0.159010,0.372188
2022-03-08,A,128.64,132.500,127.540,129.32,128.640000,132.500000,127.540000,129.320,2375482.0,-0.482485,0.005286,0.152424,0.645100
2022-03-09,A,132.85,133.940,131.190,133.26,132.850000,133.940000,131.190000,133.260,1805537.0,-0.274751,0.003086,0.445968,0.512537


In [8]:
A.groupby('ticker')['rank_volumedelta'].rolling(n_roll).corr(A['rank_ret_intraday'])

ticker  date      
A       2012-01-03         NaN
        2012-01-04         NaN
        2012-01-05         NaN
        2012-01-06         NaN
        2012-01-09         NaN
                        ...   
        2022-03-07    0.063509
        2022-03-08    0.110471
        2022-03-09    0.237759
        2022-03-10    0.270955
        2022-03-11   -0.211890
Length: 2565, dtype: float64

In [44]:
# stocks_hist['alpha'] = stocks_hist.groupby('date').tsargmax_withcond.rank(pct=True).values

In [56]:
stocks_hist

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,ret,retvol20,closeadj_sqr,closeadj_argmax,retvol20_sqr,retvol20_argmax,tsargmax_withcond,alpha
date,,,,,,,,,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0,NaN,NaN,579.557476,NaN,NaN,NaN,NaN,NaN
2012-01-04,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0,-0.007975,NaN,570.349924,NaN,NaN,NaN,NaN,NaN
2012-01-05,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0,0.022402,NaN,596.189889,NaN,NaN,NaN,NaN,NaN
2012-01-06,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0,0.010812,NaN,609.151761,NaN,NaN,NaN,NaN,NaN
2012-01-09,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0,0.026214,NaN,641.507584,4.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-03-11,ZZ,2.20,2.210,2.200,2.20,2.200000,2.210000,2.200000,2.200,407100.0,0.000000,0.006292,4.840000,3.0,0.000040,3.0,4.0,0.8
2013-03-12,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,160000.0,0.004545,0.006356,4.884100,4.0,0.000040,2.0,5.0,1.0
2013-03-13,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,308900.0,0.000000,0.006291,4.884100,3.0,0.000040,1.0,4.0,0.8


In [27]:
s_time_chunk = time.time()
stocks_hist['closeadj_argmax'] = stocks_hist.groupby('ticker').closeadj.rolling(5).apply(np.argmax, engine='cython', raw=True).values
e_time_chunk = time.time()

print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 18.187129259109497 sec


In [28]:
stocks_hist

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,ret,retvol20,op1,op2
date,,,,,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0,NaN,NaN,NaN,NaN
2012-01-04,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0,-0.007975,NaN,NaN,NaN
2012-01-05,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0,0.022402,NaN,NaN,NaN
2012-01-06,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0,0.010812,NaN,NaN,NaN
2012-01-09,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0,0.026214,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-03-11,ZZ,2.20,2.210,2.200,2.20,2.200000,2.210000,2.200000,2.200,407100.0,0.000000,0.006292,4.840000,3.0
2013-03-12,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,160000.0,0.004545,0.006356,4.884100,4.0
2013-03-13,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,308900.0,0.000000,0.006291,4.884100,3.0
